In [1]:


from archive.MS_LBM_BGK.functions import *

In [2]:
nx = 100
ny = 100
num_species = 3
nB = 15

f = np.random.random((num_species, 9, nx, ny))

phi = np.ones(num_species)
molecular_weight = np.ones(num_species)

In [3]:
def bgk_step(f, molecular_weight, phi, nB):

    #################### Before streaming ####################

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    ux_star_s, uy_star_s = calculate_u_star(CHI_sc, rho_s, rho_mix, ux_s, uy_s)

    feq = equilibrium(f, rho_s, phi, ux_star_s, uy_star_s)

    print(feq.shape)

    g_dagger_s =calculate_g_dagger(f, feq, lambda_s)

    print(g_dagger_s.shape)

    #################### After streaming ####################

    f_streamed = lattice_stream(f)

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f_streamed, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    Chi_S = post_stream_Chi_S(CHI_sc, rho_s, rho_mix)

    ux_dagger, uy_dagger, jx_s, jy_s = solve_ms_fluxes(lambda_s, Chi_S, CHI_sc, rho_s, rho_mix, ux_s, uy_s, theta=theta)

    ux_star_dagger_s, uy_star_dagger_s = calculate_u_star(CHI_sc, rho_s, rho_mix, ux_dagger, uy_dagger)

    feq_dagger = equilibrium(f_streamed, rho_s, phi, ux_star_dagger_s, uy_star_dagger_s)

    f_new = distribution_semi_implicit(feq_dagger, g_dagger_s, lambda_s)

    return f_new

In [4]:
bgk_step(f, molecular_weight, phi, nB)

(3, 9, 100, 100)
gs_shape=  (3, 9, 100, 100)
(3, 1, 3, 9, 100, 100)


array([[[[[[ 2.61399085e+00,  3.08975179e+00,  2.93131700e+00, ...,
             2.79964565e+00,  2.70631813e+00,  2.63373168e+00],
           [ 2.40461320e+00,  3.16425935e+00,  3.26633683e+00, ...,
             3.50301780e+00,  3.09624151e+00,  2.51159212e+00],
           [ 3.54219640e+00,  2.52950302e+00,  3.26399544e+00, ...,
             3.00667954e+00,  2.36719899e+00,  3.25937854e+00],
           ...,
           [ 3.51025021e+00,  2.75883010e+00,  3.47001874e+00, ...,
             3.48268727e+00,  2.81424938e+00,  3.19038742e+00],
           [ 2.49589316e+00,  2.95993526e+00,  2.88228728e+00, ...,
             3.18187222e+00,  2.98946426e+00,  3.87164644e+00],
           [ 2.96355631e+00,  2.93473020e+00,  3.55728167e+00, ...,
             2.66006662e+00,  2.62811587e+00,  2.67881499e+00]],

          [[ 4.63724831e-01,  4.40809128e-01,  6.64534929e-01, ...,
             5.29982803e-01,  8.42198681e-01,  4.45706893e-01],
           [ 1.75777147e-01,  6.56195475e-01,  3.99377071e

In [10]:
b = np.array([
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    ]
)
b[1,:] *= 2
b[2,:] *= 3
print(b)

[[ 1  2  3  4  5  6  7  8]
 [ 2  4  6  8 10 12 14 16]
 [ 3  6  9 12 15 18 21 24]]


In [16]:
delta = a[None, :] -b
print(delta)

[[  0   0   0   0   0   0   0   0]
 [ -1  -2  -3  -4  -5  -6  -7  -8]
 [ -2  -4  -6  -8 -10 -12 -14 -16]]
